In [ ]:
from pathlib import Path
import json

In [ ]:
in_data = Path("cty.dat").read_text().strip()

In [ ]:
records_list = []
current_country = None
current_region = None
callsign_mangle = []

for row in in_data.split("\n"):
    if not row.startswith(" "):
        ## this is a country name row
        ## append existing data to records
        if current_country:
            records_list.append((current_country, current_region, callsign_mangle))
        ## update with new data
        current_country = row.split(":", 1)[0]
        current_region = row.split(":  ")[3]
        callsign_mangle = []
        continue
    if row.startswith(" "):
        callsign_mangle.extend(
            list(
                filter(lambda x: "(" not in x,
                filter(lambda x: "[" not in x,
                filter(len,
                filter(lambda x: not x.startswith("="), [y.strip().replace(";", "") for y in row.strip().split(",")])
            ))))
        )

In [ ]:
records_list

In [ ]:
with open("../res/callsign_ref.json", "w") as outjson:
    outjson.write(json.dumps({
        country: {"region": region, "callsigns": [f"^{x}" for x in callsigns]} 
        for country, region, callsigns
        in filter(len, records_list)
    }, indent=4))